In [1]:
import pandas as pd
import numpy as np
from numpy import log2 as log

In [13]:
eps = np.finfo(float).eps  # it gives the smallest positive value of the float data type
print(eps)

2.220446049250313e-16


In [3]:
data=pd.read_csv("dicision tree dataset.csv")
data

,Age,Income,Gender,MaritialStatus,Buys
0,<21,High,Male,Single,No
1,<21,High,Male,Married,No
2,21-35,High,Male,Single,Yes
3,>35,Medium,Male,Single,Yes
4,>35,Low,Female,Single,Yes
5,>35,Low,Female,Married,No
6,21-35,Low,Female,Married,Yes
7,<21,Medium,Male,Single,No
8,<21,Low,Female,Married,Yes
9,>35,Medium,Female,Single,Yes


In [4]:
data.describe()

,Age,Income,Gender,MaritialStatus,Buys
count,14,14,14,14,14
unique,3,3,2,2,2
top,<21,Medium,Male,Single,Yes
freq,5,6,7,7,9


In [5]:
data['Buys'].value_counts()

Yes    9
No     5
Name: Buys, dtype: int64

In [8]:
def find_entropy(data):
  Class = data.keys()[-1]  # Class = Buys
  entropy = 0
  values = data[Class].unique()  # values = ['No', 'Yes']
  for value in values:
    prob = data[Class].value_counts()[value]/len(data[Class])  # 'No' = 5/14, 'Yes' = 9/14
    entropy += -prob*np.log2(prob)
  return entropy

In [9]:
total_Entropy = find_entropy(data)
print(total_Entropy)

0.9402859586706311


In [20]:
#Find entropy of the attribute (Each Columns)
def find_entropy_attribute(data,attribute):
    Class = data.keys()[-1]   # Class = Buys => To make the code generic, changing target variable class name
    target_variables = data[Class].unique()  # target_variables = ['No', 'Yes']
    variables = data[attribute].unique()   #This gives different features in that attribute (like 'High','Medium','Low' in Income)
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(data[attribute][data[attribute]==variable][data[Class] ==target_variable]) # num[[Income=High] && [Buys==Yes]] = 2
            den = len(data[attribute][data[attribute]==variable])  # den[Income==High] = 4
            fraction = num/(den+eps)  # fraction = 2/4 = 0.5
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(data)  # fraction2 = 4/14 = 0.2857142857142857
        entropy2 += -fraction2*entropy
    return abs(entropy2)

In [21]:
# Find the Root Node
def find_winner(data):
    #Entropy_att = []
    IG = [] # IG => Information Gain
    for key in data.keys()[:-1]:  # key in ['Age', 'Income', 'Gender', 'MaritalStatus']
    #Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(data)-find_entropy_attribute(data,key))
    return data.keys()[:-1][np.argmax(IG)]  # returns the attribute with maximum IG

In [22]:
# Find the leaf Nodes
def get_subtable(data, node, value):
    return data[data[node] == value].reset_index(drop=True)

In [29]:
# Build Tree
def buildTree(data,tree=None): 
    Class = data.keys()[-1]   # Class = Buys
    
    #Here we build our decision tree

    #Get attribute with maximum information gain
    node = find_winner(data)
    
    #Get distinct value of that attribute e.g Income is node and Low, Med and High are values
    attValue = np.unique(data[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
    #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 

    for value in attValue:
        
        subtable = get_subtable(data,node,value)  # this gives leaf node value
        clValue,counts = np.unique(subtable['Buys'],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree

In [24]:
#Check the function
tree = buildTree(data)

In [27]:
tree

{'Age': {'21-35': 'Yes',
  '<21': {'Gender': {'Female': 'Yes', 'Male': 'No'}},
  '>35': {'MaritialStatus': {'Married': 'No', 'Single': 'Yes'}}}}

In [25]:
def find_Gini_Index(data):
  Class = data.keys()[-1]
  gini = 0
  values = data[Class].unique()
  for value in values:
    prob = data[Class].value_counts()[value]/len(data[Class])
    gini += (prob*prob)
  return (1-gini)

In [26]:
Gini_Index = find_Gini_Index(data)
print(Gini_Index)

0.4591836734693877
